# Import library

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from keras.applications import vgg16
from keras.applications import resnet_v2

import numpy as np
from keras.layers import Dense, Dropout, Flatten
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model 
from IPython.display import clear_output
from keras.models import model_from_json
%matplotlib inline

# Split train date to 10 folder

In [12]:
# 1. Make dir
for i in range(0,10):
    path=f'./train/{i}'
    if (os.path.isdir(path)==False):
            os.mkdir(path)

In [13]:
# 2. Split folder
amount=0
for file in os.scandir('./train/'):
    if ( os.path.isdir(file.path)==False and file.path.endswith('.png')):
            fileDir=file.path[-5]
            newPart=f'train/{fileDir}/'
            newPath=file.path.replace('train/',newPart)
            os.rename(file.path,newPath)
            amount+=1
            print(amount)
            if amount%100==0:
                 clear_output()


# Transferlearning vgg16 model 

## Import image by ImageDataGenerators

In [12]:
train_datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest",
        preprocessing_function=vgg16.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    directory=r"./train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=15,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=r"./test",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n

Found 50001 images belonging to 10 classes.
Found 100 images belonging to 2 classes.


## Define Model

In [16]:
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)        # new FC layer, random init
x = Dense(128, activation='relu')(x)        # new FC layer, random init
x = Dropout(0.2)(x)
predictions = Dense(10, activation='softmax')(x)  # 10 labels

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False

W0208 08:51:15.057378 4592270784 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0208 08:51:15.115659 4592270784 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0208 08:51:15.151080 4592270784 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0208 08:51:15.238459 4592270784 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:42

## Complie and fit model

In [13]:
#   Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for i in range(20):
    model.fit_generator(
                                generator=train_generator,
                                steps_per_epoch=STEP_SIZE_TRAIN,
                                epochs=20
                            )
    model_structure=model.to_json()
    with open("./vgg16/model.json", "w") as json_file:
        json_file.write(model_structure)
        model.save_weights('./vgg16/weight.h5')

Epoch 1/20
3333/3333 [==============================] - 1568s 471ms/step - loss: 0.7336 - acc: 0.7649
Epoch 2/20
3333/3333 [==============================] - 1435s 430ms/step - loss: 0.7191 - acc: 0.7670
Epoch 3/20
3333/3333 [==============================] - 1636s 491ms/step - loss: 0.7148 - acc: 0.7688
Epoch 4/20
3333/3333 [==============================] - 1571s 471ms/step - loss: 0.7149 - acc: 0.7710
Epoch 5/20
3333/3333 [==============================] - 1530s 459ms/step - loss: 0.7131 - acc: 0.7679
Epoch 6/20
3333/3333 [==============================] - 1559s 468ms/step - loss: 0.7075 - acc: 0.7707
Epoch 7/20
 108/3333 [..............................] - ETA: 31:08 - loss: 0.6940 - acc: 0.7765

## Prediction

In [19]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

100/100 [==============================] - 2s 19ms/step


In [20]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print("predictions for test images" , predictions)
"".join(predictions)

predictions for test images ['9', '7', '6', '8', '6', '3', '2', '2', '0', '7', '6', '4', '0', '0', '1', '0', '3', '9', '4', '0', '4', '0', '5', '8', '9', '5', '4', '4', '6', '3', '6', '7', '2', '7', '9', '4', '8', '0', '1', '3', '0', '8', '8', '7', '7', '0', '4', '1', '7', '6', '9', '2', '4', '1', '7', '8', '2', '7', '3', '3', '1', '1', '2', '8', '0', '7', '6', '5', '8', '6', '1', '2', '9', '3', '3', '0', '4', '5', '0', '5', '8', '7', '4', '9', '8', '9', '3', '7', '8', '3', '1', '4', '0', '5', '3', '3', '6', '3', '4', '7']


'9768632207640010394040589544636727948013088770417692417827331128076586129330450587498937831405336347'

## Answers

In [ ]:
import pandas as pd
ans_df = pd.read_csv('./submission.csv')
ans_dict=dict(zip([name[-8:] for name in test_generator.filenames],predictions))
ans_df['label']=ans_df['filename'].apply(lambda name:ans_dict[name])

In [ ]:
ans_df.to_csv('./vgg16/submission.csv',index=False)

## Save and load model

In [ ]:
model_structure=model.to_json()
with open("./vgg16/model.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights('./vgg16/weight.h5')

In [8]:
json_file = open('./vgg16/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("./vgg16/weight.h5")

W0208 21:12:03.690408 4591381952 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0208 21:12:03.803282 4591381952 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0208 21:12:03.871707 4591381952 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0208 21:12:04.115999 4591381952 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_

# Transferlearning ResNet152V2 model

## Import image by ImageDataGenerators

In [5]:
train_datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest",
        preprocessing_function=resnet_v2.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=resnet_v2.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    directory=r"./train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=10,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=r"./test",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n

Found 49929 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


## Define Model

In [6]:
base_model = resnet_v2.ResNet152V2(weights='imagenet', include_top=False, input_shape=(64,64,3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)        # new FC layer, random init
x = Dense(128, activation='relu')(x)        # new FC layer, random init
x = Dropout(0.2)(x)
predictions = Dense(10, activation='softmax')(x)  # 10 labels

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Complie and fit model

In [7]:
#   Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for i in range(100):
  print(i)
  model.fit_generator(
                                generator=train_generator,
                                steps_per_epoch=STEP_SIZE_TRAIN,
                                epochs=30
                            )
  model_structure=model.to_json()
  with open("./resnet/model.json", "w") as json_file:
      json_file.write(model_structure)
  model.save_weights('./resnet/weight.h5')

0
Epoch 1/30
4992/4992 [==============================] - 214s 43ms/step - loss: 1.5695 - acc: 0.4579
Epoch 2/30
4992/4992 [==============================] - 212s 42ms/step - loss: 1.5581 - acc: 0.4614
Epoch 3/30
4992/4992 [==============================] - 207s 41ms/step - loss: 1.5444 - acc: 0.4651
Epoch 4/30
4992/4992 [==============================] - 217s 44ms/step - loss: 1.5346 - acc: 0.4712
Epoch 5/30
4992/4992 [==============================] - 206s 41ms/step - loss: 1.5258 - acc: 0.4726
Epoch 6/30
4992/4992 [==============================] - 203s 41ms/step - loss: 1.5252 - acc: 0.4735
Epoch 7/30
4992/4992 [==============================] - 193s 39ms/step - loss: 1.5130 - acc: 0.4800
Epoch 8/30
4992/4992 [==============================] - 190s 38ms/step - loss: 1.5117 - acc: 0.4812
Epoch 9/30
4992/4992 [==============================] - 193s 39ms/step - loss: 1.5103 - acc: 0.4822
Epoch 10/30
4992/4992 [==============================] - 192s 38ms/step - loss: 1.5087 - acc: 0.48

## Prediction

In [9]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

100/100 [==============================] - 6s 65ms/step


In [10]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print("predictions for test images" , predictions)
"".join(predictions)

predictions for test images ['0', '7', '4', '9', '5', '3', '0', '2', '0', '2', '1', '7', '9', '8', '8', '0', '3', '9', '7', '0', '1', '9', '2', '0', '7', '5', '2', '4', '5', '3', '1', '7', '2', '7', '1', '2', '0', '0', '1', '5', '0', '8', '8', '2', '5', '0', '5', '9', '7', '3', '9', '2', '4', '1', '7', '9', '2', '7', '5', '8', '1', '1', '0', '8', '0', '9', '2', '5', '8', '5', '9', '2', '9', '9', '5', '0', '1', '5', '0', '5', '0', '7', '1', '9', '0', '9', '5', '5', '9', '0', '1', '4', '0', '5', '5', '5', '1', '3', '7', '7']


'0749530202179880397019207524531727120015088250597392417927581108092585929950150507190955901405551377'

## Answers

In [ ]:
import pandas as pd
ans_df = pd.read_csv('./submission.csv')
ans_dict=dict(zip([name[-8:] for name in test_generator.filenames],predictions))
ans_df['label']=ans_df['filename'].apply(lambda name:ans_dict[name])

## Save and load model

In [ ]:
model_structure=loaded_model.to_json()
with open("./resnet/model.json", "w") as json_file:
    json_file.write(model_structure)
loaded_model.save_weights('./resnet/weight.h5')

NameError: ignored

In [ ]:
json_file = open('./resnet/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights("./resnet/weight.h5")